# NBA Who's Hot and Who's Not 🔥❄️

This notebook shows how to use the `nba_api` library to pull NBA player data and compare their **season average points** vs their **last 5 games average**. The idea: identify which players are currently *hot* (🔥) or *not* (❄️).

In [4]:
# Install nba_api if not already installed
%pip install nba_api pandas
import sys
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.1 MB 12.8 MB/s eta 0:00:01
   ----------------------- ---------------- 4.7/8.1 MB 12.2 MB/s eta 0:00:01
   --------------------------------- ------ 6.8/8.1 MB 11.3 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 11.3 MB/s  0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 11.7 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------------ --------------------- 3.1/7.0 MB 15.8 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 17.5 MB/s  0:00:00

   ---------------------------------------- 0/7 [pyparsing]
   ---------------------------------------- 0/7 [pyparsing]
   ----- ---------------------------------- 1/7 [pillow]
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Step 1: Import libraries and get list of players

In [21]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
import pandas as pd
import matplotlib.pyplot as plt

# Get all active players
all_players = players.get_active_players()
# print(all_players[0])  # show example structure

# Build dictionary of active players
player_dict = {p['full_name']: p['id'] for p in all_players}
len(player_dict)


572

## Step 2: Define function to compute Hot or Not

In [22]:

# Let's filter Players so they meet certain criteria
season = "2024-25"
# Minimum games played this season
min_games = 40
# Minimum minutes per game
min_mins = 20
# Game amount to determine hotness
window = 5

#playergamelog.PlayerGameLog(player_id=1630173, season='2024-25').get_data_frames()[0]

# Player list
player_list = []

# Shrink the dictionary to filter for all players that have 
# over 20 minutes for the 2024-25 season
for name, pid in player_dict.items():
    
    # Get dataframe of games for player
    gamelog = playergamelog.PlayerGameLog(player_id=pid, season='2025-26').get_data_frames()[0]
    
    # If the gamelog is empty or their are not enough minimum games
    if gamelog.empty or len(gamelog) < min_games:
        # Do not include this player
        continue

    # Get the season average of minutes per players
    season_avg_mins = gamelog["MIN"].mean()
    # If the season average is under 20 minutes
    if season_avg_mins < min_mins:
        # Do not include this player
        continue

    # Get season average and recent average
    season_avg = gamelog["PTS"].mean()
    recent_avg = gamelog.head(window)["PTS"].mean()

    # Get Hottness ratio
    ratio = recent_avg / season_avg

    status = ""
    # Get the Status
    if ratio > 1.05:
        status = "Hot"
    elif ratio < 0.95:
        status = "Cold"
    else:
        status = "Neutral"
    
    # Append to list
    player_list.append({
        "Player": name,
        "Season Avg": round(season_avg, 1),
        "Last 5 Avg": round(recent_avg, 1),
        "Ratio": round(ratio, 1),
        "Status": status
    })

# Change to Data Frame
df_pl = pd.DataFrame(player_list)

# Grab 15 players
df_pl.head(15)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

## Step 3: Filter Hot Players 🔥

In [17]:
# Filter for hot players
df_pl
hot_players = df_pl[df_pl["Status"] == "Hot"]

# display
hot_players

KeyError: 'Status'

## Step 4: Plot Hotness Ratio
You can save the table for use in your blog post.

In [23]:
plt.figure(figsize=(10,8))

# Bar colors based on status
colors = df_pl["Status"].map({"Hot": "red", "Neutral": "gray", "Not": "blue"})

# Scatter plot
plt.scatter(df_pl["Season Avg"], df_pl["Last 5 Avg"], color=colors, s=100)

# Diagonal line y = x
max_val = max(df_pl["Season Avg"].max(), df_pl["Last 5 Avg"].max()) + 5
plt.plot([0, max_val], [0, max_val], color="black", linestyle="--", label="y = Season Avg")

# Labels
plt.xlabel("Season Average Points")
plt.ylabel("Recent 5 Games Average Points")
plt.title("NBA Players: Season vs Recent Average Points")
plt.legend(["y = Season Avg", "Players"], loc="upper left")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

KeyError: 'Status'

<Figure size 1000x800 with 0 Axes>